## This code demonstrates how to apply FinBERT fine-tuned for ESG classification

> *Created by Allen Huang (HKUST) and Xiangyu Li (USC)*


## 1. Set up Fine-Tuned FinBERT Model
First check GPU selection: Select Menu "Runtime" -> Change runtime type -> select "GPU"

In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# load the FinBERT fine-tuned ESG 9-category model and the corresponding tokernizer (used for preprocessing input sentences)
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories', num_labels = 9)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')

# load model on GPU/CPU
import torch
# load model on GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
    # put the model on GPU
    finbert.to(device)
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Model loaded on:', torch.cuda.get_device_name(0))

# load model on CPU if GPU is not available
else:
    device = torch.device("cpu")
    # put the model on CPU
    finbert.to(device)
    print('No GPU available, model loaded on CPU.')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

There are 1 GPU(s) available.
Model loaded on: Tesla T4


## 2. Load sample data set (225 sentences = 25 x 9 categories)

In [ ]:
import pandas as pd

# import sample data
data = pd.read_csv('https://github.com/allenhhuang/FinBERT/raw/main/FinBERT%20test%20sample.csv').dropna(subset = ['Sentence', 'Label']).reset_index(drop = True)

# 9 categories and labels
label_map = {'Climate Change': 'CC',
             'Natural Capital': 'NC',
             'Pollution & Waste': 'PW',
             'Human Capital': 'HC',
             'Product Liability': 'PL',
             'Community Relations': 'CR',
             'Corporate Governance': 'CG',
             'Business Ethics & Values': 'BE',
             'Non-ESG': 'N'}

label_map_reverse = {'CC': 'Climate Change',
             'NC': 'Natural Capital',
             'PW': 'Pollution & Waste',
             'HC': 'Human Capital',
             'PL': 'Product Liability',
             'CR': 'Community Relations',
             'CG': 'Corporate Governance',
             'BE': 'Business Ethics & Values',
             'N': 'Non-ESG'}

# print the distribution of categories in sample data and first 5 observations
print(data['Label'].value_counts())
print('First 5 observations in sample data:')
data.head(5)

Label
PL    25
CG    25
HC    25
CC    25
BE    25
CR    25
PW    25
N     25
NC    25
Name: count, dtype: int64
First 5 observations in sample data:


,Sentence,Label
0,"Importantly, our measure of MRSL compliance is...",PL
1,"It assists NIKE, Inc. in accomplishing its obj...",CG
2,"Internally, our Women of Nike & Friends Networ...",HC
3,"In Belgium, at our Converse European Logistics...",CC
4,The Company will not make political contributi...,BE


### 2.1. OPTION 1: Output each category's probability

In [ ]:
# define a function to output the probability of each category
from scipy.special import softmax
import numpy as np

def augmented_nlp(sentence):
  '''input any sentence, output FinBERT label, Predicted Label score/probability,
     and the probabilities that the sentence belongs to each category'''
  # preprocess the sentence
  input = tokenizer(sentence, return_tensors = 'pt', padding = True).to(device)
  # finbert raw output
  output = finbert(**input)[0]
  # convert the output to probabilities corresponding to each category
  probs = softmax(output.cpu().detach().numpy(), axis = 1)
  # get the numeric label with the highest probability
  value = np.argmax(probs, axis = -1)
  # get the textual label
  label = finbert.config.id2label[value[0]]
  # get the highest probability
  score = np.max(probs, axis = -1)

  return {'label': label,
          'score': score[0],
          'probabilities': probs[0].tolist()}

# demonstrate the output
print('augmented nlp output: ', augmented_nlp('For 2002, our total net emissions were approximately 60 million metric tons of CO2 equivalents for all businesses and operations we have ﬁnancial interests in, based on its equity share in those businesses and operations.'))

augmented nlp output:  {'label': 'Climate Change', 'score': 0.9955656, 'probabilities': [0.9955655932426453, 0.0006281687528826296, 0.0010522982338443398, 0.00031542990473099053, 0.00038357896846719086, 0.0002711373381316662, 0.0009126343647949398, 0.00028890068642795086, 0.000582217937335372]}


In [ ]:
# Process sample data and download full output in a CSV file

# define the output structure
header = ['No.', 'Sentence', 'Label', 'FinBERT label', 'FinBERT score', 'Correct label'] + list(finbert.config.id2label.values())
result = []

# loop through the sentences
for i, row in data.iterrows():
  # sentence number
  id = str(i+1)
  print('Sentence number: ' + id)

  # sentence
  sentence = row['Sentence']
  print('Sentence: ' + sentence)

  # use the augmented_nlp function to process the sentence, get finbert label, score, and probabilities
  finbert_output = augmented_nlp(sentence)

  # label and score
  manual_label = row['Label']
  finbert_label = label_map[finbert_output['label']]
  finbert_score = finbert_output['score']
  print('Manual label: ' + label_map_reverse[manual_label])
  print('FinBERT label: ' + finbert_output['label'])
  print('FinBERT score: ', finbert_score)

  # probabilities
  finbert_probs = finbert_output['probabilities']

  print('-'*50)
  print()

  # combine all the output above into one row of observation
  row = [id, sentence, manual_label, finbert_label, finbert_score, 1*(manual_label==finbert_label)] + finbert_probs
  result.append(row)

# consolidate the output into a dataframe
result_df_full = pd.DataFrame(result, columns = header)

# save the output in the csv format
result_df_full.to_csv('FinBERT ESG9Class output full.csv', index = False)
# download the output to the local computer
from google.colab import files
files.download('FinBERT ESG9Class output full.csv')
# display the output
result_df_full

Sentence number: 1
Sentence: Importantly, our measure of MRSL compliance is based at the facility level and not solely on NIKE production.
Manual label: Product Liability
FinBERT label: Product Liability
FinBERT score:  0.7817407
--------------------------------------------------

Sentence number: 2
Sentence: It assists NIKE, Inc. in accomplishing its objectives by partnering with management to build and maintain effective risk management, control and governance processes.
Manual label: Corporate Governance
FinBERT label: Corporate Governance
FinBERT score:  0.9760307
--------------------------------------------------

Sentence number: 3
Sentence: Internally, our Women of Nike & Friends Network – through NikeUNITED – continues to play a significant role in retention and promotion of women through its robust, interactive programming.
Manual label: Human Capital
FinBERT label: Human Capital
FinBERT score:  0.81121886
--------------------------------------------------

Sentence number: 4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No.,Sentence,Label,FinBERT label,FinBERT score,Correct label,Climate Change,Natural Capital,Pollution & Waste,Human Capital,Product Liability,Community Relations,Corporate Governance,Business Ethics & Values,Non-ESG
0,1,"Importantly, our measure of MRSL compliance is...",PL,PL,0.781741,1,0.005837,0.008719,0.018283,0.108884,0.781741,0.001808,0.027980,0.042614,0.004134
1,2,"It assists NIKE, Inc. in accomplishing its obj...",CG,CG,0.976031,1,0.001785,0.000750,0.000673,0.002529,0.002088,0.000599,0.976031,0.006527,0.009018
2,3,"Internally, our Women of Nike & Friends Networ...",HC,HC,0.811219,1,0.002511,0.001111,0.001281,0.811219,0.017498,0.126240,0.009988,0.017754,0.012397
3,4,"In Belgium, at our Converse European Logistics...",CC,CC,0.995495,1,0.995495,0.000916,0.001224,0.000271,0.000408,0.000422,0.000548,0.000280,0.000436
4,5,The Company will not make political contributi...,BE,BE,0.968667,1,0.001151,0.001800,0.001200,0.003276,0.001506,0.002203,0.016750,0.968667,0.003447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,221,"However, we expect the future operating model ...",N,N,0.990633,1,0.000611,0.000397,0.000871,0.000824,0.001539,0.000979,0.003462,0.000684,0.990633
221,222,NIKE required suppliers to assess water-relate...,NC,NC,0.987169,1,0.003108,0.987169,0.003630,0.000524,0.001987,0.001856,0.000600,0.000832,0.000296
222,223,"Specifically, through collaboration and engage...",NC,NC,0.970531,1,0.002104,0.970531,0.017974,0.000855,0.004111,0.001953,0.000774,0.001352,0.000346
223,224,"In June 2020, we announced a new digitally emp...",N,PL,0.853872,0,0.029915,0.006763,0.004608,0.007451,0.853872,0.037358,0.007024,0.048121,0.004888


### 2.2. OPTION 2: Only Output Predicted Label



In [ ]:
# use pipeline in transformers to assemble the steps of finbert prediction
if torch.cuda.is_available(): # has GPU
  nlp = pipeline("text-classification", model = finbert, tokenizer = tokenizer, device = 0)
else: # CPU only
  nlp = pipeline("text-classification", model = finbert, tokenizer = tokenizer)


# demonstrate the output
print('nlp output: ', nlp('For 2002, our total net emissions were approximately 60 million metric tons of CO2 equivalents for all businesses and operations we have ﬁnancial interests in, based on its equity share in those businesses and operations.'))

nlp output:  [{'label': 'Climate Change', 'score': 0.9955655932426453}]


In [ ]:
# Process sample data and download predicted label

# define the output structure
header = ['No.', 'Sentence', 'Label', 'FinBERT label', 'FinBERT score', 'Correct label']
result = []

# loop through the sentences
for i, row in data.iterrows():

  # sentence number
  id = str(i+1)
  print('Sentence number: ' + id)

  # sentence
  sentence = row['Sentence']
  print('Sentence: ' + sentence)

  # use the nlp function to process the sentence, get finbert label and score
  finbert_output = nlp(sentence)[0]

  # label and score
  manual_label = row['Label']
  finbert_label = label_map[finbert_output['label']]
  finbert_score = finbert_output['score']
  print('Manual label: ' + label_map_reverse[manual_label])
  print('FinBERT label: ' + finbert_output['label'])
  print('FinBERT score: ', finbert_score)

  print('-'*50)
  print()

  # combine all the output above into one row of observation
  row = [id, sentence, manual_label, finbert_label, finbert_score, 1*(manual_label==finbert_label)]
  result.append(row)

# consolidate the output into a dataframe
result_df_label = pd.DataFrame(result, columns = header)

# save the output in the csv format
result_df_label.to_csv('FinBERT ESG9Class output label.csv', index = False)
# download the output to the local computer
from google.colab import files
files.download('FinBERT ESG9Class output label.csv')
# display the output
result_df_label

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Sentence number: 1
Sentence: Importantly, our measure of MRSL compliance is based at the facility level and not solely on NIKE production.
Manual label: Product Liability
FinBERT label: Product Liability
FinBERT score:  0.7817407250404358
--------------------------------------------------

Sentence number: 2
Sentence: It assists NIKE, Inc. in accomplishing its objectives by partnering with management to build and maintain effective risk management, control and governance processes.
Manual label: Corporate Governance
FinBERT label: Corporate Governance
FinBERT score:  0.976030707359314
--------------------------------------------------

Sentence number: 3
Sentence: Internally, our Women of Nike & Friends Network – through NikeUNITED – continues to play a significant role in retention and promotion of women through its robust, interactive programming.
Manual label: Human Capital
FinBERT label: Human Capital
FinBERT score:  0.8112188577651978
----------------------------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No.,Sentence,Label,FinBERT label,FinBERT score,Correct label
0,1,"Importantly, our measure of MRSL compliance is...",PL,PL,0.781741,1
1,2,"It assists NIKE, Inc. in accomplishing its obj...",CG,CG,0.976031,1
2,3,"Internally, our Women of Nike & Friends Networ...",HC,HC,0.811219,1
3,4,"In Belgium, at our Converse European Logistics...",CC,CC,0.995495,1
4,5,The Company will not make political contributi...,BE,BE,0.968667,1
...,...,...,...,...,...,...
220,221,"However, we expect the future operating model ...",N,N,0.990633,1
221,222,NIKE required suppliers to assess water-relate...,NC,NC,0.987169,1
222,223,"Specifically, through collaboration and engage...",NC,NC,0.970531,1
223,224,"In June 2020, we announced a new digitally emp...",N,PL,0.853872,0


## 3. Evaluate FinBERT performance (only if you already have label)

In [ ]:
# evaluate the FinBERT performance
from sklearn.metrics import accuracy_score, precision_score, recall_score

# select which output to evaluate: result_df_full or result_df_label
# eval = result_df_full
eval = result_df_label

print('Accuracy score: ', accuracy_score(eval['Label'], eval['FinBERT label']))
print('Precision score: ', precision_score(eval['Label'], eval['FinBERT label'], average = 'macro'))
print('Recall score: ', recall_score(eval['Label'], eval['FinBERT label'], average = 'macro'))

Accuracy score:  0.9333333333333333
Precision score:  0.9334194814316596
Recall score:  0.9333333333333333


## 4. Try a single sentence

In [ ]:
# paste any sentence here within the quotation marks
sentence = 'For 2002, our total net emissions were approximately 60 million metric tons of CO2 equivalents for all businesses and operations we have ﬁnancial interests in, based on its equity share in those businesses and operations.'
results = nlp(sentence)

# get FinBERT labels and scores
finbert_output = nlp(sentence)[0]
finbert_label = finbert_output['label']
finbert_score = finbert_output['score']

print('Sentence: ' + sentence)
print('FinBERT label: ' + finbert_label)
print('FinBERT score: ', finbert_score)

Sentence: For 2002, our total net emissions were approximately 60 million metric tons of CO2 equivalents for all businesses and operations we have ﬁnancial interests in, based on its equity share in those businesses and operations.
FinBERT label: Climate Change
FinBERT score:  0.9955655932426453


## 5. Try an unlabeled dataset

In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# load the FinBERT fine-tuned ESG 9-category model and the corresponding tokernizer
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories', num_labels = 9)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')

# load model on GPU/CPU
import torch
# load model on GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
    # put the model on GPU
    finbert.to(device)
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Model loaded on:', torch.cuda.get_device_name(0))

# load model on CPU if GPU is not available
else:
    device = torch.device("cpu")
    # put the model on CPU
    finbert.to(device)
    print('No GPU available, model loaded on CPU.')

import pandas as pd

# import sample data
data = pd.read_csv('https://github.com/allenhhuang/FinBERT/raw/main/FinBERT%20test%20sample.csv').dropna(subset = ['Sentence']).reset_index(drop = True)

# 9 categories and labels
label_map = {'Climate Change': 'CC',
             'Natural Capital': 'NC',
             'Pollution & Waste': 'PW',
             'Human Capital': 'HC',
             'Product Liability': 'PL',
             'Community Relations': 'CR',
             'Corporate Governance': 'CG',
             'Business Ethics & Values': 'BE',
             'Non-ESG': 'N'}

# use pipeline in transformers to assemble the steps of finbert prediction
if torch.cuda.is_available(): # has GPU
  nlp = pipeline("text-classification", model = finbert, tokenizer = tokenizer, device = 0)
else: # CPU only
  nlp = pipeline("text-classification", model = finbert, tokenizer = tokenizer)

# define the output structure
header = ['No.', 'Sentence', 'FinBERT label', 'FinBERT score']
result = []

# loop through the sentences
for i, row in data.iterrows():

  # sentence number
  id = str(i+1)
  # sentence
  sentence = row['Sentence']

  # use the nlp function to process the sentence, get finbert label and score
  finbert_output = nlp(sentence)[0]

  # label and score
  finbert_label = label_map[finbert_output['label']]
  finbert_score = finbert_output['score']

  # combine all the output above into one row of observation
  row = [id, sentence, finbert_label, finbert_score]
  result.append(row)

# consolidate the output into a dataframe
result_df_label = pd.DataFrame(result, columns = header)

# save the output in the csv format
result_df_label.to_csv('FinBERT ESG9Class prediction.csv', index = False)
# download the output to the local computer
from google.colab import files
files.download('FinBERT ESG9Class prediction.csv')
# display the output
result_df_label

There are 1 GPU(s) available.
Model loaded on: Tesla T4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,No.,Sentence,FinBERT label,FinBERT score
0,1,"Importantly, our measure of MRSL compliance is...",PL,0.781741
1,2,"It assists NIKE, Inc. in accomplishing its obj...",CG,0.976031
2,3,"Internally, our Women of Nike & Friends Networ...",HC,0.811219
3,4,"In Belgium, at our Converse European Logistics...",CC,0.995495
4,5,The Company will not make political contributi...,BE,0.968667
...,...,...,...,...
220,221,"However, we expect the future operating model ...",N,0.990633
221,222,NIKE required suppliers to assess water-relate...,NC,0.987169
222,223,"Specifically, through collaboration and engage...",NC,0.970531
223,224,"In June 2020, we announced a new digitally emp...",PL,0.853872
